In [2]:
#匯入套件
import json
import fnmatch
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random
from tqdm import tqdm

#Models
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

/Users/andy/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [12]:
#匯入training data
dic={} 
with open("/Users/andy/Desktop/2017學習數據分析黑客松/hack_training.csv", 'r') as f:  
    for line in f: 
        course=line.split(',')[0] 
        userid =line.split(',')[1] 
        if course not in dic: 
            dic[course]=[userid]
        else: 
            dic[course].append(userid)
dic.pop('course_id')

['user_id']

In [ ]:
#載入所有資料
data = []
error_data = []
for i in range(4,8):
    for name in tqdm(os.listdir("/Users/andy/Desktop/2017學習數據分析黑客松/hack_log_201"+str(i))):
        with open('/Users/andy/Desktop/2017學習數據分析黑客松/hack_log_201'+str(i)+"/"+name, 'r') as f:
            for line in f:
                try:
                     if json.loads(line)['context']['course_id'] in dic:
                            if str(json.loads(line)['context']['user_id']) in dic[json.loads(line)['context']['course_id']]:
                                data.append(json.loads(line))
                except:
                    error_data.append((line,f)) 

 87%|████████▋ | 2578/2979 [19:06<02:58,  2.25it/s]

In [10]:
#匯入training data
train = pd.read_csv("/Users/andy/Desktop/2017學習數據分析黑客松/hack_training.csv")

In [14]:
#全部的data取出training data要的資料
num=1
dict1={}
col=["['context']['course_id']","['context']['session']","['context']['user_id']","['event_type']","['event']['id']","['event']['currentTime']","['time']"]
for a in tqdm(range(len(data1))):
    content=[]
    try:
        content.append(data1[a]['context']['course_id'])
    except:
        content.append(np.nan)
    try:
        content.append(data1[a]['context']['session'])
    except:
        content.append(np.nan)
    try:
        content.append(data1[a]['context']['user_id'])
    except:
        content.append(np.nan)
    try:
        content.append(data1[a]['event_type'])
    except:
        content.append(np.nan)
    try:
        content.append(json.loads(data1[0]['event'])['id'])
    except:
        content.append(np.nan)
    try:
        content.append(json.loads(data1[a]['event'])['currentTime'])
    except:
        content.append(np.nan)
    try:
        content.append(data1[a]['time'])
    except:
        content.append(np.nan)
    dict1[num]=content
    num+=1

100%|██████████| 100000/100000 [00:04<00:00, 21588.25it/s]


In [38]:
#轉成dataframe格式
df=pd.DataFrame.from_dict(dict1,orient='index')
df = df.rename(columns={0:'course_id',1:'session',2:'user_id',3:'event_type',4:'video_id',5:'currenttime',6:'time'})
df.to_csv('all.csv')

In [ ]:
#匯入qusetion2資料
train = pd.read_csv("/Users/andy/Desktop/2017學習數據分析黑客松/hack_question02.csv")
df = pd.read_csv('all.csv')

In [40]:
df2 = df[df['user_id']!=1653]
# df3 = df2[df2['user_id']!=2472]

In [ ]:
# all_uid
uid = df['user_id'].value_counts().index

In [42]:
# all_course_id
k=train['course_id'].value_counts()
classname=k.index

In [ ]:
#切天數
df['date']=df['time'].str.split("T").str[0]
#切時間
df['time1']=df['time'].str.split("T").str[1]
df['time1']=df['time1'].str.split(".").str[0]

In [ ]:
#把每個欄位匯入
num=1
fdic={}
u_dic={}
c_dic={}
dc_dic={}
vc_dic={}
at_dic={}
sc_dic={}
se_dic={}
pl_dic={}
sevi_dic={}
spva_dic={}

#training的所有user
for e_u in tqdm(uid):
    k=df[df['user_id']==e_u]['course_id'].value_counts().index
    #training裡面user的所有課程
    for each in k:
        dn=df[(df['user_id']==e_u) & (df['course_id']==each)]
        #user_id
        u_dic[num]=e_u
        #course_id
        c_dic[num]=each
        #datecount
        dc_dic[num]=len(dn['date'].value_counts())
        #video_count
        vc_dic[num]=len(dn['video_id'].value_counts())
        #session_count
        se_dic[num]=len(dn['session'].value_counts())
        #影片觀看次數
        a=0
        session = []
        for len_session in range(len(dn['session'].value_counts())):
            if dn['session'] not in session:
                session.append(dn['session'])
                a+=len(dn['video_id'].value_counts())
        sevi_dic[num] = a
        #pause_count
        pl_dic[num]=len(dn[dn['event_type']=='pause_video'])
        #speed_change_count
        sc_dic[num]=len(dn[dn['event_type']=='speed_change_video'])
        #speed_change_value
        speed = 0
        speed+=(dn['new_speed']-dn['old_speed']).sum()
        spva_dic[num]=speed
        
        #all_time
        total=0
        p=[]
        n=dn.index
        for a in n:
            if dn['event_type'][a]=='play_video':
                p.append(a)
            elif dn['event_type'][a]=='pause_video' and p!=[]:
                if dn['session'][a] == dn['session'][p[-1]]:
                    d1=datetime.datetime.strptime(dn['time1'][a],"%H:%M:%S")
                    d2=datetime.datetime.strptime(dn['time1'][p[-1]],"%H:%M:%S")
                    diff=relativedelta(d1,d2)
                    total+=int(diff.minutes)*60+int(diff.seconds)
                p=[]
        at_dic[num]=total                  
        num+=1
fdic['user_id']=u_dic
fdic['course_id']=c_dic
fdic['datecount']=dc_dic
fdic['videocount']=vc_dic
fdic['session_videocount']=se_dic
fdic['totaltime']=at_dic
fdic['sessioncount']=se_dic
fdic['pause_video_count']=pl_dic
fdic['speed_change_count']=sc_dic
fdic['speed_change_value']=spva_dic
fdf=pd.DataFrame.from_dict(fdic)

In [ ]:
#以該課程所有人中最多觀看影片數量最多的人為基準，來做影片完成率
c_id=fdf['course_id'].value_counts().index
courseperid={}
ddm={}
num=1
for each in c_id:
    courseperid[each]=fdf[fdf['course_id']==each]['videocount'].max()
courseperid
for a in range(1,len(fdf)+1):
    ddm[num]=fdf['videocount'][a]/courseperid[fdf['course_id'][a]]
    num+=1
fdic['ddm']=ddm
fdf=pd.DataFrame.from_dict(fdic)
fdf.to_csv("finall.csv")

In [10]:
# 建模

#載入資料
df = pd.read_csv("finall.csv")
train = pd.read_csv("/Users/andy/Desktop/2017學習數據分析黑客松/hack_training.csv")

In [11]:
#資料清理
df = df.drop('Unnamed: 0', 1)
df3 = df.merge(train, on=('user_id','course_id'), left_index=True, right_index=True)
df3['target1'] = df3['target'].map({'T':1,'F':0})
df4 = df3.drop('target', 1)
df5 = df4.drop('target1', 1)
df6 = df5.drop('ddm',1)

In [12]:
q2 = pd.read_csv('finallfinall.csv')
q3 = q2.drop('Unnamed: 0',1)

In [15]:
#KNN
X_train, X_test, y_train, y_test = train_test_split(df6, df4['target1'], test_size=0.3, random_state=1000)
knn = KNeighborsClassifier()
X_test['course_id'] = LabelEncoder().fit_transform(X_test['course_id'])
X_train['course_id'] = LabelEncoder().fit_transform(X_train['course_id'])
knn.fit(X_train, y_train)

#預測準確率
y_predict = knn.predict(X_test)
print('Misclassified samples: %d' % (y_test != y_predict).sum())
print('Accuracy: %.2f' % accuracy_score(y_test, y_predict))

Misclassified samples: 797
Accuracy: 0.78


/Users/andy/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/andy/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [16]:
#載入目標學生資料
q3['course_id'] = LabelEncoder().fit_transform(q3['course_id'])
target = knn.predict(q3)
final =pd.DataFrame({"user_id":q2['user_id'],"course_id":q2['course_id'],'target':target})
final['target'] = final['target'].map({1:'T',0:'F'})

#輸出最終結果
final.to_csv('q2.csv')